## 1. Data Acquisition

#### 1A. Get Climatogical Data from PAGASA Weather Station Data

From the downloaded PDF files, these series of code extracts the useful data, clean it and export as a CSV

In [7]:
# ----- GET FILE PATHS OF PAGASA Station Data -----------------------------

from pathlib import Path
import glob

# Specify the relative folder path
relative_folder_path = 'data\PAG-ASA Climatologial Normals per Station'  # Change this to your folder path

# Get the absolute path of the folder
absolute_folder_path = Path(relative_folder_path).resolve()

# List all files in the folder and get their relative paths
file_names = [Path(file) for file in glob.glob(f"{absolute_folder_path}/**/*", recursive=True) if Path(file).is_file()]

print("DONE")

DONE


In [8]:
# EXTRACT THE DATA FROM EACH PDF FILE & CLEAN IT
import pdfquery
import re
import pandas as pd

import tabula
# from tabulate import tabulate

month_mapping = {
    'JAN': 'January',
    'FEB': 'February',
    'MAR': 'March',
    'APR': 'April',
    'MAY': 'May',
    'JUN': 'June',
    'JUL': 'July',
    'AUG': 'August',
    'SEP': 'September',
    'OCT': 'October',
    'NOV': 'November',
    'DEC': 'December'
}


def convert_DMS_to_decimal(dms_str):
# Extract the direction (N/S/E/W)
    direction = dms_str[-1]
    
    # Remove the direction and split the DMS string into components
    dms_str = dms_str[:-1]
    # dms_parts = dms_str.split('o')
    dms_parts = re.split(r'[o°]', dms_str)
    degrees = float(dms_parts[0])
    
    minutes_seconds = dms_parts[1].split('\'')
    minutes = float(minutes_seconds[0])
    seconds = float(minutes_seconds[1].replace('"', ''))

    # Convert DMS to decimal degrees
    decimal_degrees = degrees + (minutes / 60) + (seconds / 3600)
    
    # Adjust for direction
    if direction in ['S', 'W']:
        decimal_degrees = -decimal_degrees
    
    return decimal_degrees
    
def split_column(df, column):
    split_cols = df[column].str.split(r'\s+', expand=True)
    split_cols.columns = [f"{column}_{i+1}" for i in range(split_cols.shape[1])]
    return split_cols

def convert_column_name(col_name):
    return float(str(col_name).replace('_', '.'))


# Initialize dataframes 
pagasa_stations_data = []
pagasa_stations_climatological_normals = pd.DataFrame()

for path in file_names:
    print(path)
    #read the PDF
    pdf = pdfquery.PDFQuery(path)
    pdf.load()

    #convert to xml
    pdf.tree.write('customers.xml', pretty_print = True)

    # access the data using coordinates
    station_name = pdf.pq('LTTextLineHorizontal:contains("STATION:")').text()[9:]
    station_data = {
        'station_name': station_name,
        'latitude': convert_DMS_to_decimal(
            pdf.pq('LTTextLineHorizontal:contains("LATITUDE: ")').text()[10:]
        ),
        'longitude': convert_DMS_to_decimal(
            pdf.pq('LTTextLineHorizontal:contains("LONGITUDE: ")').text()[11:]
        ),
        'elevation_meters': pdf.pq('LTTextLineHorizontal:contains("ELEVATION: ")').text()[11:].replace("m", "").strip()
        
    }

    # store to a csv file
    tabula.convert_into(path, "output.csv", output_format="csv", pages='all')

    #read data from csv file and store to dataframe
    df = pd.read_csv('output.csv', skiprows=8, header=None)

    #clean the extracted data
    # Iterate over all columns and split if the column is of type object (string)
    for column in df.select_dtypes(include=['object']).columns:
        split_cols = split_column(df, column)
        df = df.drop(columns=[column]).join(split_cols)
    
    # Sort the columns based on their numeric equivalent
    sorted_columns = sorted(df.columns, key=convert_column_name)

    # Reorder the DataFrame columns
    df = df[sorted_columns]

    df.dropna(how='all', axis=1, inplace=True) 
    df.columns = [
        "month",
        "rainful_mm","number_of_rainy_days",
        "max_temp_C","min_temp_C","ave_temp_C","dry_bulb_C","wet_bulb_C","dew_point_C",
        "vapor_press_mbs",
        "rel_hum_perc",
        "mean_sea_lvl_pressure_mbs",
        "wind_dr_16pt",
        "wind_spd_mps",
        "cloud_atm_okta",
        "days_with_thunderstorm",
        "days_with_lightning"
    ]
    
    df = pd.melt(df,id_vars = 'month', var_name = 'attribute', value_name = 'value')
    
    # add columns
    df['location_scope'] = 'pagasa_station'
    df['location_name'] = station_name
    df['time_breakdown'] = df['month'].map(month_mapping)
    df['breakdown'] = 'month'

    df = df[['location_scope','location_name','time_breakdown','breakdown','attribute','value']]
    
    
    pagasa_stations_data.append(station_data)
    pagasa_stations_climatological_normals = pd.concat([pagasa_stations_climatological_normals, df], ignore_index=True)



D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Normals per Station\ALABAT.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Normals per Station\AMBULONG.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Normals per Station\APARRI.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Normals per Station\BAGUIO.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Normals per Station\BALER RADAR.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Normals per Station\BASCO RADAR.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Normals per Station\BORONGAN.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Normals per Station\BUTUAN CITY.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Normals per Station\CABANATUAN.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Normals per Station\CALAPAN.pdf
D:\Programming\ph_climate_analysis\data\PAG-AS

In [9]:
# ----------- EXPORT THE CLEAN DATA TO A CSV FILE

import csv
# Field names
fieldnames = ['station_name', 'latitude', 'longitude', 'elevation_meters']

# Writing to CSV file
with open('data\pagasa_stations_data.csv', mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    
    # Write the header & data
    writer.writeheader()
    writer.writerows(pagasa_stations_data)


# Writing to CSV file
pagasa_stations_climatological_normals.to_csv('data\pagasa_stations_climatological_normals.csv',index=False)

### 1B. Get Climatological Extremes per stations

In [61]:
# ----- GET FILE PATHS OF PAGASA Station Data -----------------------------

from pathlib import Path
import glob

# Specify the relative folder path
relative_folder_path = 'data\PAG-ASA Climatologial Extremes per Station'  # Change this to your folder path

# Get the absolute path of the folder
absolute_folder_path = Path(relative_folder_path).resolve()

# List all files in the folder and get their relative paths
file_names = [Path(file) for file in glob.glob(f"{absolute_folder_path}/**/*", recursive=True) if Path(file).is_file()]

print(file_names)
print("DONE")

[WindowsPath('D:/Programming/ph_climate_analysis/data/PAG-ASA Climatologial Extremes per Station/Alabat.pdf'), WindowsPath('D:/Programming/ph_climate_analysis/data/PAG-ASA Climatologial Extremes per Station/Ambulong.pdf'), WindowsPath('D:/Programming/ph_climate_analysis/data/PAG-ASA Climatologial Extremes per Station/Aparri.pdf'), WindowsPath('D:/Programming/ph_climate_analysis/data/PAG-ASA Climatologial Extremes per Station/Baguio.pdf'), WindowsPath('D:/Programming/ph_climate_analysis/data/PAG-ASA Climatologial Extremes per Station/Baler.pdf'), WindowsPath('D:/Programming/ph_climate_analysis/data/PAG-ASA Climatologial Extremes per Station/Basco.pdf'), WindowsPath('D:/Programming/ph_climate_analysis/data/PAG-ASA Climatologial Extremes per Station/Borongan.pdf'), WindowsPath('D:/Programming/ph_climate_analysis/data/PAG-ASA Climatologial Extremes per Station/Butuan.pdf'), WindowsPath('D:/Programming/ph_climate_analysis/data/PAG-ASA Climatologial Extremes per Station/Cabanatuan-CLSU.pdf')

In [62]:
# EXTRACT THE DATA FROM EACH PDF FILE & CLEAN IT
import pdfquery
import re
import pandas as pd

import tabula
# from tabulate import tabulate

month_mapping = {
    'JAN': 'January',
    'FEB': 'February',
    'MAR': 'March',
    'APR': 'April',
    'MAY': 'May',
    'JUN': 'June',
    'JUL': 'July',
    'AUG': 'August',
    'SEP': 'September',
    'OCT': 'October',
    'NOV': 'November',
    'DEC': 'December'
}


def convert_DMS_to_decimal(dms_str):
# Extract the direction (N/S/E/W)
    direction = dms_str[-1]
    
    # Remove the direction and split the DMS string into components
    dms_str = dms_str[:-1]
    # dms_parts = dms_str.split('o')
    dms_parts = re.split(r'[o°]', dms_str)
    degrees = float(dms_parts[0])
    
    minutes_seconds = dms_parts[1].split('\'')
    minutes = float(minutes_seconds[0])
    seconds = float(minutes_seconds[1].replace('"', ''))

    # Convert DMS to decimal degrees
    decimal_degrees = degrees + (minutes / 60) + (seconds / 3600)
    
    # Adjust for direction
    if direction in ['S', 'W']:
        decimal_degrees = -decimal_degrees
    
    return decimal_degrees
    
def split_column(df, column):
    split_cols = df[column].str.split(r'(?<!-)\s+', expand=True)
    split_cols.columns = [f"{column}_{i+1}" for i in range(split_cols.shape[1])]
    return split_cols

def convert_column_name(col_name):
    return float(str(col_name).replace('_', '.'))


# Initialize dataframes 
pagasa_stations_data = []
pagasa_stations_climatological_extreme = pd.DataFrame()

for path in file_names:
    print(path)
    #read the PDF
    pdf = pdfquery.PDFQuery(path)
    pdf.load()

    #convert to xml
    pdf.tree.write('customers.xml', pretty_print = True)

    # access the data using coordinates
    station_name = pdf.pq('LTTextLineHorizontal:contains("STATION:")').text()[9:]
    station_data = {
        'station_name': station_name,
        'latitude': convert_DMS_to_decimal(
            pdf.pq('LTTextLineHorizontal:contains("LATITUDE: ")').text()[10:]
        ),
        'longitude': convert_DMS_to_decimal(
            pdf.pq('LTTextLineHorizontal:contains("LONGITUDE: ")').text()[11:]
        ),
        'elevation_meters': pdf.pq('LTTextLineHorizontal:contains("ELEVATION: ")').text()[11:].replace("m", "").strip()
        
    }

    # store to a csv file
    tabula.convert_into(path, "output.csv", output_format="csv", pages='all')

    #read data from csv file and store to dataframe
    df = pd.read_csv('output.csv', header=None)
    
    last_index = df[df[0] == 'ANNUAL'].index[0]
    first_index = df[df[1] == 'HIGH'].index[0] + 1
    df = df.iloc[first_index:last_index]
    
    #clean the extracted data
    # Iterate over all columns and split if the column is of type object (string)
    for column in df.select_dtypes(include=['object']).columns:
        split_cols = split_column(df, column)
        df = df.drop(columns=[column]).join(split_cols)
    

    # Sort the columns based on their numeric equivalent
    sorted_columns = sorted(df.columns, key=convert_column_name)
    
    # Reorder the DataFrame columns
    df = df[sorted_columns]
    
    df.dropna(how='all', axis=1, inplace=True)
    
    df.to_csv('data\output_test.csv',index=False)
    try:
        df.columns = [
            "month",
            "temp_high","temp_high_date", 
            "temp_low","temp_low_date",
            "rainfall_mm","rainfall_mm_date",
            "strongest_winds_spd","strongest_winds_spd_dir","strongest_winds_date",
            "sea_lvl_pressure_mbs_high", "sea_lvl_pressure_mbs_high_date",
            "sea_lvl_pressure_mbs_low", "sea_lvl_pressure_mbs_low_date"
        ]
    except Exception as e:
        print(repr(e))
        print(df)

    df["strongest_winds"] = df['strongest_winds_spd'] + " at " + df['strongest_winds_spd_dir']
    df = df.drop(columns=['strongest_winds_spd','strongest_winds_spd_dir'])
    
    df = pd.melt(df,id_vars = 'month', var_name = 'attribute', value_name = 'value')

    # Separate the attribute and date columns
    df["date"] = df.apply(lambda row: row["value"] if "date" in row["attribute"] else None, axis=1)
    df["attribute"] = df["attribute"].str.replace("_date", "")

    # Forward fill the dates
    df["date"] = df["date"].ffill()

    # Filter out the rows where value is a date
    df = df[~df["attribute"].str.contains("date")]

    # add columns
    df['location_scope'] = 'pagasa_station'
    df['location_name'] = station_name
    df['time_breakdown'] = df['month'].map(month_mapping)
    df['breakdown'] = 'month'

    df = df[['location_scope','location_name','time_breakdown','breakdown','attribute','value','date']]

    df = df[~(df.value.isnull())]

    def combine_rows(group):
        try:
            # Get the value where date is blank
            value_no_date = group.loc[(group['value'].str.contains("-")), 'value'].values[0]

            # Get the value from the date column
            value_with_date = group.loc[~(group['value'].str.contains("-")), 'value'].values[0]
            # Update the value column
            group['value'] = value_no_date
            group.loc[group['date'] != '', 'value'] = value_with_date
            return group.iloc[0]
        except Exception as e:
            print(group)

    # Group by the specified columns and apply the custom function
    df = df.groupby(['location_scope', 'location_name', 'time_breakdown', 'breakdown', 'attribute']).apply(combine_rows).reset_index(drop=True)
    
    
    pagasa_stations_data.append(station_data)
    pagasa_stations_climatological_extreme = pd.concat([pagasa_stations_climatological_extreme, df], ignore_index=True)



D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Extremes per Station\Alabat.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Extremes per Station\Ambulong.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Extremes per Station\Aparri.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Extremes per Station\Baguio.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Extremes per Station\Baler.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Extremes per Station\Basco.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Extremes per Station\Borongan.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Extremes per Station\Butuan.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Extremes per Station\Cabanatuan-CLSU.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA Climatologial Extremes per Station\Calapan.pdf
D:\Programming\ph_climate_analysis\data\PAG-ASA 

In [63]:
# ----------- EXPORT THE CLEAN DATA TO A CSV FILE

import csv

# Writing to CSV file
pagasa_stations_climatological_extreme.to_csv('data\pagasa_stations_climatological_extremes.csv',index=False)